In [94]:
path_dir = "../"

In [97]:
import random 
import numpy as np
import nltk
import codecs
import tqdm

In [98]:
"""importing the filess"""

sentences = [x.strip().split() for x in tqdm.tqdm(codecs.open(F'{path_dir}/Data/test_sentences.txt','rU','utf-8').readlines())]
housein_sentences = [x.strip().split() for x in tqdm.tqdm(codecs.open(F'{path_dir}/Data/housein_tests.txt','rU','utf-8').readlines())]
events = [x.strip().split() for x in tqdm.tqdm(codecs.open(F'{path_dir}/Data/events.txt','rU','utf-8').readlines())]

100%|██████████| 10/10 [00:00<00:00, 166440.63it/s]


In [99]:
"""normalizing the text"""

from __future__ import unicode_literals
from hazm import *

normalizer = Normalizer()
# normalizing by sentence to get a better result
sentences_normalized_by_sentence = [normalizer.normalize(' '.join(x)) for x in tqdm.tqdm(sentences)]
housein_sentences_normalized_by_sentence = [normalizer.normalize(' '.join(x)) for x in tqdm.tqdm(housein_sentences)]
# normalizing by words
sentences_normalized_by_word = [[normalizer.normalize(y) for y in x.strip().split()] for x in tqdm.tqdm(sentences_normalized_by_sentence)]
housein_sentences_normalized_by_word = [[normalizer.normalize(y) for y in x.strip().split()] for x in tqdm.tqdm(housein_sentences_normalized_by_sentence)]

events_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(events)]

100%|██████████| 10/10 [00:00<00:00, 5181.35it/s]


In [100]:
"""tokenizing the text"""

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
sentences_no_punctuation = [tokenizer.tokenize(' '.join(sent)) for sent in sentences_normalized_by_word ]
temp_sentences_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(sentences_no_punctuation)]
sentences_tokens = [[(word[0] if len(word) else '') for word in sent] for sent in tqdm.tqdm(temp_sentences_tokens)]

housein_sentences_no_punctuation = [tokenizer.tokenize(' '.join(sent)) for sent in housein_sentences_normalized_by_word ]
housein_temp_sentences_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(housein_sentences_no_punctuation)]
housein_sentences_tokens = [[(word[0] if len(word) else '') for word in sent] for sent in tqdm.tqdm(housein_temp_sentences_tokens)]

temp_events_tokens = [[word_tokenize(sent) for sent in sents] for sents in tqdm.tqdm(events_normalized)]
events_tokens = [[(word[0] if len(word) else '') for word in sent] for sent in tqdm.tqdm(temp_events_tokens)]

100%|██████████| 10/10 [00:00<00:00, 84222.97it/s]


In [101]:
"""lemmatization of the text"""

lemmatizer = Lemmatizer()

sentences_lemmatied = [[lemmatizer.lemmatize(word) for word in sent if len(word) != 0] for sent in tqdm.tqdm(sentences_tokens)]
housein_sentences_lemmatied = [[lemmatizer.lemmatize(word) for word in sent if len(word) != 0] for sent in tqdm.tqdm(housein_sentences_tokens)]
events_lemmatied = [[lemmatizer.lemmatize(word) for word in sent if len(word) != 0] for sent in tqdm.tqdm(events_tokens)]

100%|██████████| 10/10 [00:00<00:00, 72565.81it/s]


In [102]:
from zeep import Client
from requests.auth import HTTPBasicAuth
from requests import Session
from zeep.transports import Transport

In [103]:
# address of FarsNet's web services
wsdl_sense_service = 'http://nlp.sbu.ac.ir:8180/WebAPI/services/SenseService?WSDL'
wsdl_synset_service = 'http://nlp.sbu.ac.ir:8180/WebAPI/services/SynsetService?WSDL'


# username and token needed for authentication. You can get this token by signing up on http://farsnet.nlp.sbu.ac.ir
username = '12356'
token = 'd428eab3-3b91-11eb-8a1e-080027d731c1'

# connecting client
session = Session()
session.auth = HTTPBasicAuth(username, token)
client_sense_service = Client(wsdl_sense_service, transport=Transport(session=session))
client_synset_service = Client(wsdl_synset_service, transport=Transport(session=session))

In [21]:
"""getting the synset ids of the lemmatized text"""

sentences_synset_id = [[client_synset_service.service.getSynsetsByWord(token, 'EXACT', word) for word in sent] for sent in tqdm.tqdm(sentences_lemmatied)]

100%|██████████| 9/9 [02:34<00:00, 17.13s/it]


In [37]:
"""getting the synset ids of the lemmatized text"""

housein_sentences_synset_id = [[client_synset_service.service.getSynsetsByWord(token, 'EXACT', word) for word in sent] for sent in tqdm.tqdm(housein_sentences_lemmatied)]

100%|██████████| 7/7 [01:17<00:00, 11.11s/it]


In [105]:
"""getting the synset ids of the lemmatized text"""

events_synset_id = [[client_synset_service.service.getSynsetsByWord(token, 'EXACT', word) for word in sent] for sent in tqdm.tqdm(events_lemmatied)]

100%|██████████| 10/10 [01:50<00:00, 11.10s/it]


In [40]:
"""extracting the synset ids of the lemmatized text"""

sentences_synset_id_list = [[[synset.id for synset in word] for word in sent] for sent in tqdm.tqdm(sentences_synset_id)]
housein_sentences_synset_id_list = [[[synset.id for synset in word] for word in sent] for sent in tqdm.tqdm(housein_sentences_synset_id)]
events_synset_id_list = [[[synset.id for synset in word] for word in sent] for sent in tqdm.tqdm(events_synset_id)]

100%|██████████| 10/10 [00:00<00:00, 30066.70it/s]


In [115]:
def count_similar_events(synset_id_input, events_synset_id_list_input):
    
    """this function compares the id of words in sentences with id of words in events"""

    count_list = []
    for event_synset_id_list in events_synset_id_list_input:
        count = 0
        for word_synset_id_list in event_synset_id_list:
            if synset_id_input in word_synset_id_list:
                count += 1
        count_list.append(count)

    return count_list


def count_similar_sentences(sentences_synset_id_list_input, events_synset_id_list_input):

    """this function creates a list of similarity scores which contains similarity between each event and each word"""

    count_list = []
    for sentence_synset_id_list in sentences_synset_id_list_input:
        count_list_sentence = []
        for word_synset_id_list in sentence_synset_id_list:
            count_list_word = []
            for word_synset_id in word_synset_id_list:
                count_list_word.append(count_similar_events(word_synset_id, events_synset_id_list_input))
            count_list_sentence.append(count_list_word)
        count_list.append(count_list_sentence)

    return count_list


def calculate_word_event_similarity(similarity_list_input):

    """this function calculates the similarity score of each word with each event"""

    similarity_list_output = []
    for sentence in similarity_list_input:
        sentence_similarity_list = []
        for word in sentence:
            word_similarity_list = np.array([0,0,0,0,0,0,0,0,0,0])
            for id in word:
                word_similarity_list += id
            sentence_similarity_list.append(word_similarity_list)
        similarity_list_output.append(sentence_similarity_list)
    
    return similarity_list_output
    

def calculate_sentence_event_similarity(similarity_list_input):

    """this function calculates the similarity score of each sentence with each event"""
    
    similarity_list_output = []
    for sentence in similarity_list_input:
        sentence_similarity_list = np.array([0,0,0,0,0,0,0,0,0,0])
        for word in sentence:
            for id in word:
                sentence_similarity_list += id
        similarity_list_output.append(sentence_similarity_list)
    
    return similarity_list_output

In [109]:
similarity_list = count_similar_sentences(sentences_synset_id_list, events_synset_id_list)
housein_similarity_list = count_similar_sentences(housein_sentences_synset_id_list, events_synset_id_list)

In [116]:
default_word_event_similarity = calculate_word_event_similarity(similarity_list)
default_sentence_event_similarity = calculate_sentence_event_similarity(similarity_list)

housein_word_event_similarity = calculate_word_event_similarity(housein_similarity_list)
housein_sentence_event_similarity = calculate_sentence_event_similarity(housein_similarity_list)

In [111]:
def find_main_span(word_event_similarity_list_input):

    """this function finds the top score word of the sentence to find the main span and related desired event"""

    main_span_list = []
    for sentence in word_event_similarity_list_input:
        main_span_list.append(np.argmax([word[np.argmax(word)] for word in sentence]))
    return main_span_list

def convert_word_pos_to_span(sentences_main_word_index_input, sentences_input, sentences_tokens_input):

    """this function converts the word index to its span in the sentence"""

    span_list = []
    for i in range(len(sentences_input)):
        initial_index = ' '.join(sentences_input[i]).find(sentences_tokens_input[i][sentences_main_word_index_input[i]])
        span_list.append([initial_index, initial_index + len(sentences_input[i][sentences_main_word_index_input[i]])])
    return span_list

In [112]:
sentences_main_word_index = find_main_span(default_word_event_similarity)
housein_sentences_main_word_index = find_main_span(housein_word_event_similarity)

print(sentences_main_word_index)
print(housein_sentences_main_word_index)

[0, 3, 4, 2, 6, 8, 0, 6, 10]
[7, 3, 3, 6, 5, 0, 3]


In [114]:
print(convert_word_pos_to_span(sentences_main_word_index, sentences, sentences_tokens))
print(convert_word_pos_to_span(housein_sentences_main_word_index, housein_sentences, housein_sentences_tokens))

[[0, 8], [11, 18], [21, 26], [11, 17], [27, 33], [40, 46], [0, 5], [33, 38], [51, 54]]
[[38, 42], [14, 19], [10, 15], [36, 39], [31, 37], [0, 6], [10, 14]]
